# Prepare CNN Data

In [1]:
# TODOS
# 1. If sepate numpys, can loose a dimension -- as wont be all stacked

## Setup

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AY0e-g4Lce5HT2vPfkmR0gTSMQsmmoXKNJ7Vdp_7RrRABlifhg3lX_LZot4



Successfully saved authorization token.


In [4]:
import numpy as np
import geetools
from geetools import ui, cloud_mask
import os, datetime
import config as cf
import pandas as pd
import itertools

cloud_mask_landsatSR = cloud_mask.landsatSR()
cloud_mask_sentinel2 = cloud_mask.sentinel2()

In [5]:
KERNEL_SIZE = 224
SURVEY_NAME = 'DHS'

## Functions

In [6]:
def survey_to_fc(survey_df):
    '''
    Convert pandas dataframe of survey locations to a feature collection. 
    
    Inputs:
        survey_df: pandas dataframe of survey locations. Function assumes 
                   the dataframe contains (1) latitude, (2) longitude and
                   (3) uid variables. Assumes coordinates in WGS84.
    Returns:
        (feature collection)
    '''
    
    survey_fc_list = []
    
    n_rows = survey_df.shape[0]
    for i in range(0, n_rows):
        survey_df_i = survey_df.iloc[[i]]

        f_i = ee.Feature(ee.Geometry.Point([survey_df_i['longitude'].iloc[0], 
                                            survey_df_i['latitude'].iloc[0]]), 
                         {'uid': survey_df_i['uid'].iloc[0]})

        survey_fc_list.append(f_i)
        
    survey_fc = ee.FeatureCollection(survey_fc_list)
    
    return survey_fc

def normalized_diff(values1, values2):
    '''
    Normalized Difference Value

    Input:  values1, values2 (must be same dimensions)

    Output: np array
    '''

    return (values2 - values1)/(values2 + values1)

def ee_to_np_daytime(f, survey_df, n_rows, out_path, b_b, g_b, r_b, nir_b, single_bs):
    '''
    Transforms feature collection from neighborhood array to np array for landsat 8

    Input:  
      f (features)
      n_rows (number of features)

    Output: np array
    '''
    
    for i in range(0, n_rows):
        survey_uid = survey_df['uid'].iloc[i]
        
        f_i = f[i]['properties']
        
        brgb_l = [np.array(f_i[b_b]), np.array(f_i[g_b]), np.array(f_i[r_b])]
        brgb_np = np.stack(brgb_l, axis=-1)
        np.save(os.path.join(out_path, 'BRGB' + "_" + survey_uid + '.npy'), brgb_np)
        
        bndvi_l = normalized_diff(np.array(f_i[nir_b]), np.array(f_i[r_b]))
        bndvi_np = np.expand_dims(bndvi_l, axis=2)
        bndvi_np = np.repeat(bndvi_np, 3, -1)
        np.save(os.path.join(out_path, 'BNDVI' + "_" + survey_uid + '.npy'), bndvi_np)
                        
        SINGLE_BANDS_ALL = single_bs.copy()
        SINGLE_BANDS_ALL.append(nir_b)
        for band_i in SINGLE_BANDS_ALL:
            
            b_l = np.array(f_i[band_i])
            b_np = np.expand_dims(b_l, axis=2)
            b_np = np.repeat(b_np, 3, -1)
            np.save(os.path.join(out_path, band_i + "_" + survey_uid + '.npy'), b_np)

    return "Done"

def prep_cnn_np(survey_df,
                satellite,
                kernel_size,
                year,
                out_path):
    '''
    Creates numpy arrays for CNN

    Input:  df - pandas dataframe
            lat_name - name of latitude variable in df
            lon_name - name of longitude variable in df
    Output: geopandas dataframe
    '''

    survey_fc = survey_to_fc(survey_df)
    
    # Grab satellite and reduce it
        
    # l7 ----------------------------------------------------------------
    if satellite == "l7":
        
        # Bands
        b_b = 'B1'
        g_b = 'B2' 
        r_b = 'B3' 
        nir_b = 'B4'
        single_bs = ['B5', 'B6', 'B7']
        
        BANDS = single_bs.copy()
        BANDS.append(b_b)
        BANDS.append(g_b)
        BANDS.append(r_b)
        BANDS.append(nir_b)

        # Scale
        SCALE = 30
        
        # Year
        year_use = year
        
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('LANDSAT/LC07/C01/T1_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_landsatSR)\
            .median()\
            .multiply(0.0001)
    
    # l8 ----------------------------------------------------------------
    if satellite == "l8":
                
        # Bands
        b_b = 'B2'
        g_b = 'B3' 
        r_b = 'B4' 
        nir_b = 'B5'
        single_bs = ['B1', 'B6', 'B7', 'B10', 'B11']
        
        BANDS = single_bs.copy()
        BANDS.append(b_b)
        BANDS.append(g_b)
        BANDS.append(r_b)
        BANDS.append(nir_b)
                
        # Scale
        SCALE = 30
        
        # Year
        # landsat 8 starts in April 2013; if year is less than
        # 2014, use 2014 as year (to ensure have year before and after)
        if year < 2014:
            year_use = 2014
        else:
            year_use = year
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_landsatSR)\
            .median()\
            .multiply(0.0001)
        
    # s2 ----------------------------------------------------------------
    if satellite == "s2":
        
        # Bands
        b_b = 'B2'
        g_b = 'B3' 
        r_b = 'B4' 
        nir_b = 'B8'
        single_bs = ['B5', 'B6', 'B7', 'B8A', 'B9', 'B11', 'B12']
     
        BANDS = single_bs.copy()
        BANDS.append(b_b)
        BANDS.append(g_b)
        BANDS.append(r_b)
        BANDS.append(nir_b)
        
        # Scale
        SCALE = 10
        
        # Year
        # sentinel starts in March 2017; juse use 2018
        year_use = 2018
                    
        year_plus = year_use + 1
        year_minus = year_use - 1
        
        year_minus_str = str(year_minus) + '-01-01'
        year_plus_str = str(year_plus) + '-12-31'
        
        image = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterDate(year_minus_str, year_plus_str)\
            .map(cloud_mask_sentinel2)\
            .median()\
            .multiply(0.0001)

    # Select Bands  
    image = image.select(BANDS)
        
    # Image to neighborhood array
    list = ee.List.repeat(1, KERNEL_SIZE)
    lists = ee.List.repeat(list, KERNEL_SIZE)
    kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

    arrays = image.neighborhoodToArray(kernel)
    
    # Extract values from GEE    
    values_ee = arrays.sample(
      region = survey_fc, 
      scale = SCALE,
      tileScale = 8
    )
    
    dict_ee = values_ee.getInfo()
    
    # Convert values to numpy array
    n_rows = survey_df.shape[0]
    f = dict_ee['features']
    
    if satellite in ['l7', 'l8', 's2']:
        out = ee_to_np_daytime(f, survey_df, n_rows, out_path, b_b, g_b, r_b, nir_b, single_bs)

    return "Done"

def chunk_ids(total_length, chunk_size):
    n_numbers = np.ceil(total_length / chunk_size)
    n_numbers = int(n_numbers)
    
    chunk_ids = list(range(0,n_numbers)) * chunk_size
    chunk_ids.sort()
    chunk_ids = chunk_ids[:total_length]
    
    return chunk_ids

In [7]:
## Implement

In [8]:
survey_df = pd.read_csv(os.path.join(cf.SECURE_DATA_DIRECTORY, 'Data', SURVEY_NAME, 'FinalData - PII', 'GPS_uid_crosswalk.csv'))
survey_df = survey_df[survey_df.most_recent_survey == True]
survey_df = survey_df[survey_df.country_code == 'PK']
#survey_df = survey_df.head(50)
CHUNK_SIZE = 10

In [9]:
survey_df['chunk_id'] = chunk_ids(survey_df.shape[0], CHUNK_SIZE)

In [ ]:
for year_i in list(np.unique(survey_df.year)):
    
    survey_df_yeari = survey_df[survey_df['year'] == year_i]
    
    for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):

        print(chunk_i)

        survey_df_i = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

        l8_result_i = prep_cnn_np(survey_df_i,
                                  satellite = 'l8',
                                  kernel_size = KERNEL_SIZE,
                                  year = year_i,
                                  out_path = os.path.join(cf.GOOGLEDRIVE_DIRECTORY, 
                                                          'Data', 
                                                          SURVEY_NAME, 
                                                          'FinalData',
                                                          'Individual Datasets',
                                                          'cnn_l8',
                                                          'npy'))


0
